In [517]:
import sys
sys.path.append("C:\\Users\josep\Documents\Anaconda\pkgs")
import help_tools
import wave_tools
import cv2
import h5py
from matplotlib import cm
import radar_tools
import numpy as np
from wave_tools import ConstructWave, surface_core, peak_tracking
from wave_tools import fft_interpolate, fft_interface, Spectrum
from wave_tools import SpectralAnalysis
import matplotlib.pyplot as plt
from help_tools import plotting_interface
from wave_tools import surface_core, fft_interface, dispersionRelation
from help_tools import plotting_interface, polar_coordinates, convolutional_filters, moving_average
from scipy.optimize import least_squares
from scipy.optimize import minimize
from scipy.optimize import curve_fit

In [518]:
sys.path.append("C:\\Users\josep\Desktop\PHYC40900_Project TP\Data_NEW")

In [519]:
fileName ='shearing_curr_res_7.5_dt_1.0_T_1200_U_1.0exp(0.5z)+0.05_psi_0_smax_30_0_surf3d.hdf5'

In [520]:
f = h5py.File('C:\\Users\josep\Desktop\PHYC40900_Project TP\Data_NEW\shearing_curr_res_7.5_dt_1.0_T_1200_U_1.0exp(0.5z)+0.05_psi_0_smax_30_0_surf3d.hdf5', 'r')

In [521]:
psi = 0

In [522]:
x = f.get('x')
y = f.get('y')
z = f.get('z')
t = f.get('t')
k = f.get('k')
U = f.get('U')
Uk = f.get('Uk')
eta = f.get('eta')
x = np.array(x)
y = np.array(y)
z = np.array(z)
t = np.array(t)
k = np.array(k)
U = np.array(U)
Uk = np.array(Uk)
eta = np.array(eta)

In [523]:
dx = 7.5 
dy = 7.5
dt = 1

In [524]:
depth, rows, cols = eta.shape
gridX, gridY = np.meshgrid(np.arange(1, cols+1), np.arange(1, rows+1))

# Centering and scaling
gridX = (gridX - np.mean(gridX)) * dx
gridY = (gridY - np.mean(gridY)) * dy

# ?? dont really know why youve to rescale the meshgrid of x and y

In [525]:
gridX

array([[-247.5, -240. , -232.5, ...,  232.5,  240. ,  247.5],
       [-247.5, -240. , -232.5, ...,  232.5,  240. ,  247.5],
       [-247.5, -240. , -232.5, ...,  232.5,  240. ,  247.5],
       ...,
       [-247.5, -240. , -232.5, ...,  232.5,  240. ,  247.5],
       [-247.5, -240. , -232.5, ...,  232.5,  240. ,  247.5],
       [-247.5, -240. , -232.5, ...,  232.5,  240. ,  247.5]])

In [526]:
gridY

array([[-247.5, -247.5, -247.5, ..., -247.5, -247.5, -247.5],
       [-240. , -240. , -240. , ..., -240. , -240. , -240. ],
       [-232.5, -232.5, -232.5, ..., -232.5, -232.5, -232.5],
       ...,
       [ 232.5,  232.5,  232.5, ...,  232.5,  232.5,  232.5],
       [ 240. ,  240. ,  240. , ...,  240. ,  240. ,  240. ],
       [ 247.5,  247.5,  247.5, ...,  247.5,  247.5,  247.5]])

In [527]:
maxFrequency = 4.0 
# Approximate max  frequency to consider (converted to max wavenumber below) [rad/sec]

dk = 2*np.pi/(dx*min(eta.shape[0], eta.shape[1]))
# % wavenumber resolution of spectrum in each spatial window 
# (not strictly true if dx ~= dy, but value only needs to be approximate in practice)

kW = 1*dk
# Half width of wavenumbers bins [rad/m]

In [528]:
dk

0.012503851357571316

In [548]:
wavenumbers = np.arange(3 * dk, 0.305, dk)
wavenumbers

# ?? dont know why the wavenumbers are like this 
# ?? can they be changed 

array([0.03751155, 0.05001541, 0.06251926, 0.07502311, 0.08752696,
       0.10003081, 0.11253466, 0.12503851, 0.13754236, 0.15004622,
       0.16255007, 0.17505392, 0.18755777, 0.20006162, 0.21256547,
       0.22506932, 0.23757318, 0.25007703, 0.26258088, 0.27508473,
       0.28758858, 0.30009243])

In [549]:
frequencyLimits = [0.01,20.0]
# frequency limits for masking the spectrum [min max], rad/sec

Ux_limits = [-1.0, 1.0]
# Ux current limits [m/s]

Uy_limits = [-1.0, 1.0]
# Uy current limts [m/s]

U_res = 0.1
# Current step size [m/s]

In [550]:
# The following OPTIONAL parameters involve post-processing of the Doppler shifts:
# SNR_filter: whether to use a signal-to-noise filter (false by default)
SNR_filter = 0;

SNR_threshold = np.sqrt(1)
# SNR_threshold: threshold signal-to-noise value for above filter (set to 2.0 by default)

In [551]:
# Peak_filter: whether to use a multiple peaks filter (false by default)
Peak_filter = 0;

# Peak_threshold: peak threshold of maximum value (0.5 by default)
Peak_threshold = 0.5;

# Outlier_filter: whether to use an outlier filter (quartile-based) (false by default)
Outlier_filter = 0;

# (optional): whether to include 2nd harmonic of the spectrum in the fit (false by default)
include2ndHarmonic = 0;

# (optional): whether to do the fit in log space (false by default)
logFlag = [];

# (optional) omegaWidthFun: function handle as a function of wavenumber i.e.
# @(k) f(k)...., specifying frequency width of the weighting function in
# frequency-angle space (constant wavenumber). Width is half-width 1/e^2
# point of a Gaussian function.
# omegaWidthFun = @(k) 0.4 + 0.1*k;
omegaWidthFun = lambda k: 0.05 + 0.0 * k


In [552]:
# Define the fluid & physical properties: depth, gravitational acceleration,
# surface tension coefficient
Properties = { 
    "h" : np.inf,
    "g" : 9.81,
    "T" : 0.072 / 1000
}

In [553]:
Uym, Uxm = np.meshgrid(np.arange(min(Uy_limits), max(Uy_limits) + U_res, U_res),
                       np.arange(min(Ux_limits), max(Ux_limits) + U_res, U_res))

In [554]:
Uym

array([[-1.00000000e+00, -9.00000000e-01, -8.00000000e-01,
        -7.00000000e-01, -6.00000000e-01, -5.00000000e-01,
        -4.00000000e-01, -3.00000000e-01, -2.00000000e-01,
        -1.00000000e-01, -2.22044605e-16,  1.00000000e-01,
         2.00000000e-01,  3.00000000e-01,  4.00000000e-01,
         5.00000000e-01,  6.00000000e-01,  7.00000000e-01,
         8.00000000e-01,  9.00000000e-01,  1.00000000e+00],
       [-1.00000000e+00, -9.00000000e-01, -8.00000000e-01,
        -7.00000000e-01, -6.00000000e-01, -5.00000000e-01,
        -4.00000000e-01, -3.00000000e-01, -2.00000000e-01,
        -1.00000000e-01, -2.22044605e-16,  1.00000000e-01,
         2.00000000e-01,  3.00000000e-01,  4.00000000e-01,
         5.00000000e-01,  6.00000000e-01,  7.00000000e-01,
         8.00000000e-01,  9.00000000e-01,  1.00000000e+00],
       [-1.00000000e+00, -9.00000000e-01, -8.00000000e-01,
        -7.00000000e-01, -6.00000000e-01, -5.00000000e-01,
        -4.00000000e-01, -3.00000000e-01, -2.00000000e

In [555]:
Uxm

array([[-1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00],
       [-9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
        -9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
        -9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
        -9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
        -9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
        -9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
        -9.00000000e-01, -9.00000000e-01, -9.00000000e-01],
       [-8.00000000e-01, -8.00000000e-01, -8.00000000e-01,
        -8.00000000e-01, -8.00000000e-01, -8.00000000e-01,
        -8.00000000e-01, -8.00000000e-01, -8.00000000e

In [556]:
# Assuming wavenumbers is already defined
num_wavenumbers = len(wavenumbers)

# Initialize Doppler shift variables
Ux = np.full(num_wavenumbers, np.nan)
Uy = np.full(num_wavenumbers, np.nan)

# ?? check if below are used 
UxC = np.full(num_wavenumbers, np.nan)
UyC = np.full(num_wavenumbers, np.nan)
UxLS = np.full(num_wavenumbers, np.nan)
UyLS = np.full(num_wavenumbers, np.nan)

In [557]:
SNR_max = np.full(num_wavenumbers, np.nan)

# Define some Functions

In [558]:
def generate_STCFIT_for_NSPP(STCFIT, wavenumbers=None, include2ndHarmonic=None, logFlag=None, omegaWidthFun=None,
                             SNR_filter=None, SNR_threshold=None, Peak_filter=None, Peak_threshold=None, Outlier_filter=None):

    # wavenumbers
    if wavenumbers is None:
        wavenumbers = np.arange(STCFIT['fit_param']['K_limits'][0], STCFIT['fit_param']['K_limits'][1] + 2 * np.pi / STCFIT['Windows']['sq_size_m'], 2 * np.pi / STCFIT['Windows']['sq_size_m'])

    # include2ndHarmonic
    if include2ndHarmonic is None:
        include2ndHarmonic = False

    # logFlag
    if logFlag is None:
        logFlag = False

    # omegaWidthFun
    if omegaWidthFun is None:
        dOmega = 2 * np.pi / ((STCFIT['Generic']['time_stamp'][-1] - STCFIT['Generic']['time_stamp'][0]) * 3600 * 24)
        omegaWidthFun = lambda k: dOmega + 0.1 * k

    # SNR_filter
    if SNR_filter is None:
        SNR_filter = False

    # SNR_threshold
    if SNR_threshold is None:
        SNR_threshold = 2.0

    # Peak_filter
    if Peak_filter is None:
        Peak_filter = False

    # Peak_threshold
    if Peak_threshold is None:
        Peak_threshold = 0.5

    # Outlier_filter
    if Outlier_filter is None:
        Outlier_filter = False

    STCFIT['NSPP_fit_param'] = {
        'wavenumbers': wavenumbers,
        'include2ndHarmonic': include2ndHarmonic,
        'logFlag': logFlag,
        'omegaWidthFun': omegaWidthFun,
        'SNR_filter': SNR_filter,
        'SNR_threshold': SNR_threshold,
        'Peak_filter': Peak_filter,
        'Peak_threshold': Peak_threshold,
        'Outlier_filter': Outlier_filter
    }

    return STCFIT

In [559]:
def retrieve_power_spectrum(IMG_3D, dx, dy, dt, K_limits=None, W_limits=None):
    """
    Retrieve 3D structure with power spectrum.

    Parameters:
        IMG_3D: 3D array with image sequence (X, Y, Time).
        dx: resolution in x (dim 1) [meters].
        dy: resolution in y (dim 2) [meters].
        dt: resolution in time (dim 3) [seconds].
        K_limits: [min, max] Wave-number interval to retrieve [rad/m].
        W_limits: [min, max] frequency interval to retrieve [rad/sec].

    Returns:
        Spectrum: Dictionary with power spectrum information.
    """

    if K_limits is None:
        K_limits = [-np.inf, np.inf]

    if W_limits is None:
        W_limits = [-np.inf, np.inf]

    # check limits
    if any(np.array(K_limits) < 0):
        print('retrieve_positive_spectrum: K_limits smaller than 0')

    if any(np.array(W_limits) < 0):
        print('retrieve_positive_spectrum: W_limits smaller than 0')

    # get Kx, Ky, and w axes
    Nt, Nx, Ny = IMG_3D.shape
    Kx = (2 * np.pi * 1 / dx / Nx) * np.arange(-np.ceil((Nx - 1) / 2), np.floor((Nx - 1) / 2) + 1)
    Ky = (2 * np.pi * 1 / dy / Ny) * np.arange(-np.ceil((Ny - 1) / 2), np.floor((Ny - 1) / 2) + 1)
    w = (2 * np.pi * 1 / dt / Nt) * np.arange(-np.ceil((Nt - 1) / 2), np.floor((Nt - 1) / 2) + 1)

    dKx = Kx[1] - Kx[0]
    dKy = Ky[1] - Ky[0]
    dW = w[1] - w[0]
    Norm = dKx * dKy * dW

    # get power spectra
    Spectrum_raw = np.fft.fftshift(np.fft.fftn(IMG_3D) / np.prod(IMG_3D.shape))
    power_Spectrum = np.abs(Spectrum_raw / Norm) ** 2  # normalization not necessary

    # get index boundaries
    # K domain
    ind_x = np.abs(Kx) <= K_limits[1]
    ind_y = np.abs(Ky) <= K_limits[1]
    # W domain
    ind_w = np.logical_and(w >= W_limits[0], w <= W_limits[1])

    # create 3D structure
    W_3D, Kx_3D, Ky_3D = np.meshgrid(w, Kx[ind_x], Ky[ind_y], indexing='ij')
    power_Spectrum_cut = power_Spectrum[:, ind_x, ind_y]

    # set to nan values out of K_limits
    K_3D = np.sqrt(Kx_3D**2 + Ky_3D**2)
    power_Spectrum_cut[(K_3D < K_limits[0]) | (K_3D > K_limits[1])] = np.nan

    # get original spectrum limits
    Kx_orig_limits = [Kx[0], Kx[-1]]
    Ky_orig_limits = [Ky[0], Ky[-1]]
    W_orig_limits = [w[0], w[-1]]

    # normalize spectra (sum(power_Spectrum_cut(:)) = 1)
    power_Spectrum_cut = power_Spectrum_cut / np.nansum(power_Spectrum_cut)

    # create output structure
    Spectrum = {
        'power_Spectrum': power_Spectrum_cut,
        'Kx_3D': Kx_3D,
        'Ky_3D': Ky_3D,
        'W_3D': W_3D,
        'dKx': dKx,
        'dKy': dKy,
        'dW': dW,
        'Kx_orig_limits': Kx_orig_limits,
        'Ky_orig_limits': Ky_orig_limits,
        'W_orig_limits': W_orig_limits
    }

    return Spectrum

In [560]:
from scipy.fftpack import fftn, fftshift

def retrieve_power_spectrum(IMG_3D, dx, dy, dt, K_limits=None, W_limits=None):
    # Set default values for K_limits and W_limits if not provided
    if K_limits is None:
        K_limits = [-np.inf, np.inf]
    if W_limits is None:
        W_limits = [-np.inf, np.inf]

    # Check limits
    if any(np.array(K_limits) < 0):
        print('retrieve_positive_spectrum: K_limits smaller than 0')
    if any(np.array(W_limits) < 0):
        print('retrieve_positive_spectrum: W_limits smaller than 0')

    # Get Kx, Ky, and w axes
    Nt, Nx, Ny = IMG_3D.shape
    Kx = (2 * np.pi * 1 / dx / Nx) * np.arange(-np.ceil((Nx - 1) / 2), np.floor((Nx - 1) / 2) + 1)
    Ky = (2 * np.pi * 1 / dy / Ny) * np.arange(-np.ceil((Ny - 1) / 2), np.floor((Ny - 1) / 2) + 1)
    w = (2 * np.pi * 1 / dt / Nt) * np.arange(-np.ceil((Nt - 1) / 2), np.floor((Nt - 1) / 2) + 1)

    dKx = Kx[1] - Kx[0]
    dKy = Ky[1] - Ky[0]
    dW = w[1] - w[0]
    Norm = dKx * dKy * dW

    # Get power spectra
    Spectrum_raw = fftshift(fftn(IMG_3D) / np.prod(IMG_3D.shape))
    power_Spectrum = np.abs(Spectrum_raw / Norm)**2

    # Get index boundaries
    ind_x = np.abs(Kx) <= K_limits[1]
    ind_y = np.abs(Ky) <= K_limits[1]
    ind_w = (w >= W_limits[0]) & (w <= W_limits[1])

    # Create 3D structure using meshgrid
    W_3D, Kx_3D, Ky_3D = np.meshgrid(w[ind_w], Kx[ind_x], Ky[ind_y], indexing='ij')

    # Use np.ix_ for boolean indexing to extract the desired values
    power_Spectrum_cut = power_Spectrum[np.ix_(ind_w, ind_x, ind_y)]

    # Set to NaN values outside K_limits
    K_3D = np.sqrt(Kx_3D**2 + Ky_3D**2)
    power_Spectrum_cut[(K_3D < K_limits[0]) | (K_3D > K_limits[1])] = np.nan

    # Get original spectrum limits
    Kx_orig_limits = [Kx[0], Kx[-1]]
    Ky_orig_limits = [Ky[0], Ky[-1]]
    W_orig_limits = [w[0], w[-1]]

    # Normalize spectra (sum(power_Spectrum_cut(:)) = 1)
    power_Spectrum_cut /= np.nansum(power_Spectrum_cut)

    # Create output structure
    Spectrum = {'power_Spectrum': power_Spectrum_cut,
                'Kx_3D': Kx_3D, 'Ky_3D': Ky_3D, 'W_3D': W_3D,
                'dKx': dKx, 'dKy': dKy, 'dW': dW,
                'Kx_orig_limits': Kx_orig_limits, 'Ky_orig_limits': Ky_orig_limits,
                'W_orig_limits': W_orig_limits}

    return Spectrum


In [567]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

def get_doppler_shift_velocities_nsp(Spectrum, fit_param, Properties, verboseFig=False):
    if not verboseFig:
        verboseFig = 0

    if 'include2ndHarmonic' not in fit_param:
        fit_param['include2ndHarmonic'] = 0

    if 'logFlag' not in fit_param:
        fit_param['logFlag'] = 0

    if 'SNR_filter' not in fit_param:
        fit_param['SNR_filter'] = 0

    if 'SNR_threshold' not in fit_param:
        fit_param['SNR_threshold'] = 2.0

    if 'Peak_filter' not in fit_param:
        fit_param['Peak_filter'] = 0

    if 'Peak_threshold' not in fit_param:
        fit_param['Peak_threshold'] = 0.5

    if 'Outlier_filter' not in fit_param:
        fit_param['Outlier_filter'] = 0

    out_fit = {'wavenumbers': non_loop_wavenum ,
               'Ux': np.zeros(len(non_loop_wavenum )),
               'Uy': np.zeros(len(non_loop_wavenum )),
               'SNR_max': np.zeros(len(non_loop_wavenum )),
               'Ux_2D': fit_param['Ux_2D'],
               'Uy_2D': fit_param['Uy_2D'],
               'verbose': []}

    h = Properties['h']
    g = Properties['g']
    T = Properties['T']
    omegaWidthFun = fit_param['omegaWidthFun']
    wavenumbers = fit_param['wavenumbers']

    for jj in range(len(non_loop_wavenum )):
        P = {'h': h, 'g': g, 'T': T,
             'omegaWidth': omegaWidthFun(wavenumbers[jj]),
             'kWidth': fit_param['kWidth'],
             'logFlag': fit_param['logFlag'],
             'include2ndHarmonic': fit_param['include2ndHarmonic']}

        snrG = np.zeros(fit_param['Ux_2D'].shape)

        for i in range(fit_param['Ux_2D'].shape[0]):
            for j in range(fit_param['Ux_2D'].shape[1]):
                snr_ij, _, _ = nsp_doppler_shift_extraction(Spectrum, P, wavenumbers[jj], fit_param['Ux_2D'][i, j], fit_param['Uy_2D'][i, j])
                snrG[i, j] = snr_ij

        im = np.argmax(snrG)

        if verboseFig:
            plt.figure(1002)
            plt.contourf(-fit_param['Uy_2D'], -fit_param['Ux_2D'], snrG, 20, cmap='viridis')
            plt.colorbar()
            plt.axis('image')
            plt.xlabel('$U_y$ [m/s]', fontsize=12, usetex=True)
            plt.ylabel('$U_x$ [m/s]', fontsize=12, usetex=True)
            plt.title(f'SNR: $k = {wavenumbers[jj]:.2f}$ rad/m', fontsize=14, usetex=True, fontweight='normal')
            plt.grid(True)
            plt.draw()
            plt.show()

        if np.isscalar(fit_param['Ux_2D']):
            cDoppGuess = fit_param['Ux_2D'][im]
        else:
            cDoppGuess = [fit_param['Ux_2D'][im], fit_param['Uy_2D'][im]]

        options = {'disp': False}  # Turn off display output
        Ufit = minimize(lambda U: 1 - nsp_doppler_shift_extraction(Spectrum, P, wavenumbers[jj], U[0], U[1]),
                        cDoppGuess, options=options)

        if len(cDoppGuess) > 1:
            Ufit = Ufit.x
        else:
            Ufit = [Ufit.x[0], 0]

        SNR_fit, _, _ = nsp_doppler_shift_extraction(Spectrum, P, wavenumbers[jj], Ufit[0], Ufit[1])

        out_fit['Ux'][jj] = Ufit[0]
        out_fit['Uy'][jj] = Ufit[1]
        out_fit['SNR_max'][jj] = SNR_fit

        verbose = {'k': fit_param['wavenumbers'][jj], 'SNR_2D': snrG, 'Peaks_2D': find_peaks(snrG)}
        out_fit['verbose'].append(verbose)

    # Update the output structure
    out_fit_filt = filter_outputs(out_fit, fit_param)

    return out_fit_filt


# Peak filter. Returns indices where there are multiple peaks in the SNR_2D field exceeding a threshold value.
def peak_filter(S, peak_thresh):
    inds_multiple_peaks = np.zeros(len(S), dtype=bool)

    for i in range(len(S)):
        peak_vals = S[i]['SNR_2D'][S[i]['Peaks_2D']]
        inds_multiple_peaks[i] = np.sum(peak_vals / np.max(peak_vals) > peak_thresh) > 1

    return inds_multiple_peaks


# Filter based on quartiles
def quartile_filter(x, y):
    inds_nan = np.isnan(y)

    x = x[~inds_nan]
    y = y[~inds_nan]

    pct = np.polyfit(x, y, 1)
    delCtil = y - np.polyval(pct, x)
    delCtil_srt = np.sort(delCtil)

    Q1 = np.median(delCtil_srt[:round(len(delCtil_srt) / 2)])
    Q3 = np.median(delCtil_srt[round(len(delCtil_srt) / 2):])
    IQR = Q3 - Q1
    lower_fence = Q1 - 1.5 * IQR
    upper_fence = Q3 + 1.5 * IQR

    inds_keep = np.logical_and(delCtil > lower_fence, delCtil < upper_fence)

    inds_outlier = np.ones(inds_nan.shape, dtype=bool)
    inds_outlier[~inds_nan] = inds_keep

    return inds_outlier


# Doppler Shift Loop to get Spectrum

In [568]:
non_loop_wavenum = np.arange(3 * dk, 0.305, dk)

In [569]:
for jj in range(len(wavenumbers)):
    wavenumberLimits = wavenumbers[jj] + kW * np.array([-1, 1])
    # wavenumber limits for masking the spectrum [min max], rad/sec

    # Assuming STCFIT is a dictionary and fileName, wavenumbers, include2ndHarmonic, logFlag, omegaWidthFun,
    # SNR_filter, SNR_threshold, Peak_filter, Peak_threshold, Outlier_filter are defined

    STCFIT = {'Name': [fileName]}
    STCFIT = generate_STCFIT_for_NSPP(STCFIT, wavenumbers[jj], include2ndHarmonic, logFlag,
                                  omegaWidthFun, SNR_filter, SNR_threshold, Peak_filter, Peak_threshold, Outlier_filter)
    
    fit_param = STCFIT['NSPP_fit_param']
    fit_param['kWidth'] = 4 * kW
    fit_param['Ux_2D'] = Uxm
    fit_param['Uy_2D'] = Uym
    
    # Now extract the Doppler shifts 
    Spectrum = retrieve_power_spectrum(
        eta,
        dx,
        dy,
        dt,
        wavenumberLimits,
        frequencyLimits
    )

    if fit_param['include2ndHarmonic']:
        Spectrum2 = retrieve_power_spectrum(
            eta,
            dx,
            dy,
            dt,
            2 * wavenumberLimits,
            frequencyLimits
        )

        Spectrum['power_Spectrum2'] = Spectrum2['power_Spectrum']
        Spectrum['Kx_3D2'] = Spectrum2['Kx_3D']
        Spectrum['Ky_3D2'] = Spectrum2['Ky_3D']
        Spectrum['W_3D2'] = Spectrum2['W_3D']
        
    out_DS = get_doppler_shift_velocities_nsp(Spectrum,fit_param,Properties,1);

IndexError: invalid index to scalar variable.

In [544]:
fit_param

{'wavenumbers': 0.03751155407271395,
 'include2ndHarmonic': 0,
 'logFlag': [],
 'omegaWidthFun': <function __main__.<lambda>(k)>,
 'SNR_filter': 0,
 'SNR_threshold': 1.0,
 'Peak_filter': 0,
 'Peak_threshold': 0.5,
 'Outlier_filter': 0,
 'kWidth': 0.050015405430285265,
 'Ux_2D': array([[-1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00, -1.00000000e+00, -1.00000000e+00],
        [-9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
         -9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
         -9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
         -9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
         -9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
 

In [510]:
Spectrum

{'power_Spectrum': array([[[           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan, 5.37111022e-08, ...,
          1.65455438e-08,            nan,            nan],
         [           nan, 2.08081123e-07, 1.64922946e-07, ...,
          4.58629124e-08, 9.42634105e-08,            nan],
         ...,
         [           nan, 8.80588885e-08, 4.23201431e-08, ...,
          1.77155678e-07, 2.21407793e-07,            nan],
         [           nan,            nan, 1.56392982e-08, ...,
          5.80982900e-08,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan]],
 
        [[           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan, 5.26794284e-08, ...,
          1.67886184e-08,            nan,       

In [488]:
Spectrum["power_Spectrum"].shape

(598, 51, 51)

In [489]:
np.nanmax(Spectrum["power_Spectrum"])

0.006205604765687458

In [490]:
np.nanmax(Spectrum["power_Spectrum"][0,:,:])

8.376509019565453e-08

# Doppler Velocities

In [496]:
out_DS = get_doppler_shift_velocities_nsp(Spectrum,fit_param,Properties,1);

TypeError: object of type 'numpy.float64' has no len()

In [501]:
fit_param['wavenumbers']

0.03751155407271395

In [514]:
fit_param['Ux_2D'].shape[0]

21

In [511]:
fit_param

{'wavenumbers': 0.03751155407271395,
 'include2ndHarmonic': 0,
 'logFlag': [],
 'omegaWidthFun': <function __main__.<lambda>(k)>,
 'SNR_filter': 0,
 'SNR_threshold': 1.0,
 'Peak_filter': 0,
 'Peak_threshold': 0.5,
 'Outlier_filter': 0,
 'kWidth': 0.050015405430285265,
 'Ux_2D': array([[-1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00, -1.00000000e+00, -1.00000000e+00],
        [-9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
         -9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
         -9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
         -9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
         -9.00000000e-01, -9.00000000e-01, -9.00000000e-01,
 

In [241]:
from scipy.fftpack import fftn, fftshift

def retrieve_power_spectrum(IMG_3D, dx, dy, dt, K_limits=None, W_limits=None):
    # Set default values for K_limits and W_limits if not provided
    if K_limits is None:
        K_limits = [-np.inf, np.inf]
    if W_limits is None:
        W_limits = [-np.inf, np.inf]

    # Check limits
    if any(np.array(K_limits) < 0):
        print('retrieve_positive_spectrum: K_limits smaller than 0')
    if any(np.array(W_limits) < 0):
        print('retrieve_positive_spectrum: W_limits smaller than 0')

    # Get Kx, Ky, and w axes
    Nt, Nx, Ny = IMG_3D.shape
    Kx = (2 * np.pi * 1 / dx / Nx) * np.arange(-np.ceil((Nx - 1) / 2), np.floor((Nx - 1) / 2) + 1)
    Ky = (2 * np.pi * 1 / dy / Ny) * np.arange(-np.ceil((Ny - 1) / 2), np.floor((Ny - 1) / 2) + 1)
    w = (2 * np.pi * 1 / dt / Nt) * np.arange(-np.ceil((Nt - 1) / 2), np.floor((Nt - 1) / 2) + 1)

    dKx = Kx[1] - Kx[0]
    dKy = Ky[1] - Ky[0]
    dW = w[1] - w[0]
    Norm = dKx * dKy * dW

    # Get power spectra
    Spectrum_raw = fftshift(fftn(IMG_3D) / np.prod(IMG_3D.shape))
    power_Spectrum = np.abs(Spectrum_raw / Norm)**2

    # Get index boundaries
    ind_x = np.abs(Kx) <= K_limits[1]
    ind_y = np.abs(Ky) <= K_limits[1]
    ind_w = (w >= W_limits[0]) & (w <= W_limits[1])

    # Create 3D structure using meshgrid
    W_3D, Kx_3D, Ky_3D = np.meshgrid(w[ind_w], Kx[ind_x], Ky[ind_y], indexing='ij')
    
    # Use boolean indexing to extract the desired values
    power_Spectrum_cut = power_Spectrum[ind_w, ind_x, ind_y]

    # Set to NaN values outside K_limits
    K_3D = np.sqrt(Kx_3D**2 + Ky_3D**2)
    power_Spectrum_cut[(K_3D < K_limits[0]) | (K_3D > K_limits[1])] = np.nan

    # Get original spectrum limits
    Kx_orig_limits = [Kx[0], Kx[-1]]
    Ky_orig_limits = [Ky[0], Ky[-1]]
    W_orig_limits = [w[0], w[-1]]

    # Normalize spectra (sum(power_Spectrum_cut(:)) = 1)
    power_Spectrum_cut /= np.nansum(power_Spectrum_cut)

    # Create output structure
    Spectrum = {'power_Spectrum': power_Spectrum_cut,
                'Kx_3D': Kx_3D, 'Ky_3D': Ky_3D, 'W_3D': W_3D,
                'dKx': dKx, 'dKy': dKy, 'dW': dW,
                'Kx_orig_limits': Kx_orig_limits, 'Ky_orig_limits': Ky_orig_limits,
                'W_orig_limits': W_orig_limits}

    return Spectrum

# test

In [242]:
eta.shape

(1200, 67, 67)

In [243]:
# Get Kx, Ky, and w axes
Nt, Nx, Ny = eta.shape
Kx = (2 * np.pi * 1 / dx / Nx) * np.arange(-np.ceil((Nx - 1) / 2), np.floor((Nx - 1) / 2) + 1)
Ky = (2 * np.pi * 1 / dy / Ny) * np.arange(-np.ceil((Ny - 1) / 2), np.floor((Ny - 1) / 2) + 1)
w = (2 * np.pi * 1 / dt / Nt) * np.arange(-np.ceil((Nt - 1) / 2), np.floor((Nt - 1) / 2) + 1)

dKx = Kx[1] - Kx[0]
dKy = Ky[1] - Ky[0]
dW = w[1] - w[0]
Norm = dKx * dKy * dW

In [244]:
dKx

0.012503851357571316

In [245]:
dW

0.005235987755982663

In [214]:
Norm

8.186273060646456e-07

In [215]:
Kx

array([-0.41262709, -0.40012324, -0.38761939, -0.37511554, -0.36261169,
       -0.35010784, -0.33760399, -0.32510014, -0.31259628, -0.30009243,
       -0.28758858, -0.27508473, -0.26258088, -0.25007703, -0.23757318,
       -0.22506932, -0.21256547, -0.20006162, -0.18755777, -0.17505392,
       -0.16255007, -0.15004622, -0.13754236, -0.12503851, -0.11253466,
       -0.10003081, -0.08752696, -0.07502311, -0.06251926, -0.05001541,
       -0.03751155, -0.0250077 , -0.01250385,  0.        ,  0.01250385,
        0.0250077 ,  0.03751155,  0.05001541,  0.06251926,  0.07502311,
        0.08752696,  0.10003081,  0.11253466,  0.12503851,  0.13754236,
        0.15004622,  0.16255007,  0.17505392,  0.18755777,  0.20006162,
        0.21256547,  0.22506932,  0.23757318,  0.25007703,  0.26258088,
        0.27508473,  0.28758858,  0.30009243,  0.31259628,  0.32510014,
        0.33760399,  0.35010784,  0.36261169,  0.37511554,  0.38761939,
        0.40012324,  0.41262709])

In [216]:
# Get power spectra
Spectrum_raw = fftshift(fftn(eta) / np.prod(eta.shape))
power_Spectrum = np.abs(Spectrum_raw / Norm)**2


In [217]:
power_Spectrum

array([[[0.00040279, 0.00040793, 0.00042073, ..., 0.00042246,
         0.00040946, 0.00040328],
        [0.00041976, 0.00042473, 0.0004357 , ..., 0.00044328,
         0.00042796, 0.00042044],
        [0.00045858, 0.00046229, 0.00047428, ..., 0.00048668,
         0.00046972, 0.00046067],
        ...,
        [0.00046067, 0.00046972, 0.00048668, ..., 0.00047428,
         0.00046229, 0.00045858],
        [0.00042044, 0.00042796, 0.00044328, ..., 0.0004357 ,
         0.00042473, 0.00041976],
        [0.00040328, 0.00040946, 0.00042246, ..., 0.00042073,
         0.00040793, 0.00040279]],

       [[0.00040358, 0.00040891, 0.00042029, ..., 0.00042204,
         0.00040947, 0.00040321],
        [0.00041955, 0.00042396, 0.00043643, ..., 0.000442  ,
         0.00042835, 0.00042088],
        [0.00045778, 0.00046224, 0.00047361, ..., 0.0004861 ,
         0.00046907, 0.00046065],
        ...,
        [0.00046172, 0.00047069, 0.00048707, ..., 0.0004741 ,
         0.00046269, 0.00045875],
        [0.0

In [267]:
wavenumberLimits = wavenumbers[0] + kW * np.array([-1, 1])

In [268]:
wavenumberLimits

array([0.0250077 , 0.05001541])

In [269]:
wavenumberLimits

array([0.0250077 , 0.05001541])

In [270]:
K_limits = wavenumberLimits

W_limits = frequencyLimits

In [271]:
# Get index boundaries
ind_x = np.abs(Kx) <= K_limits[1]
ind_y = np.abs(Ky) <= K_limits[1]
ind_w = (w >= W_limits[0]) & (w <= W_limits[1])

In [272]:
ind_w

array([False, False, False, ...,  True,  True,  True])

In [273]:
# Create 3D structure using meshgrid
W_3D, Kx_3D, Ky_3D = np.meshgrid(w[ind_w], Kx[ind_x], Ky[ind_y], indexing='ij')

In [274]:
Kx_3D

array([[[-0.05001541, -0.05001541, -0.05001541, ..., -0.05001541,
         -0.05001541, -0.05001541],
        [-0.03751155, -0.03751155, -0.03751155, ..., -0.03751155,
         -0.03751155, -0.03751155],
        [-0.0250077 , -0.0250077 , -0.0250077 , ..., -0.0250077 ,
         -0.0250077 , -0.0250077 ],
        ...,
        [ 0.0250077 ,  0.0250077 ,  0.0250077 , ...,  0.0250077 ,
          0.0250077 ,  0.0250077 ],
        [ 0.03751155,  0.03751155,  0.03751155, ...,  0.03751155,
          0.03751155,  0.03751155],
        [ 0.05001541,  0.05001541,  0.05001541, ...,  0.05001541,
          0.05001541,  0.05001541]],

       [[-0.05001541, -0.05001541, -0.05001541, ..., -0.05001541,
         -0.05001541, -0.05001541],
        [-0.03751155, -0.03751155, -0.03751155, ..., -0.03751155,
         -0.03751155, -0.03751155],
        [-0.0250077 , -0.0250077 , -0.0250077 , ..., -0.0250077 ,
         -0.0250077 , -0.0250077 ],
        ...,
        [ 0.0250077 ,  0.0250077 ,  0.0250077 , ...,  

In [275]:
K_limits[0]

0.025007702715142632

In [276]:
ind_w

array([False, False, False, ...,  True,  True,  True])

In [277]:
Kx

array([-0.41262709, -0.40012324, -0.38761939, -0.37511554, -0.36261169,
       -0.35010784, -0.33760399, -0.32510014, -0.31259628, -0.30009243,
       -0.28758858, -0.27508473, -0.26258088, -0.25007703, -0.23757318,
       -0.22506932, -0.21256547, -0.20006162, -0.18755777, -0.17505392,
       -0.16255007, -0.15004622, -0.13754236, -0.12503851, -0.11253466,
       -0.10003081, -0.08752696, -0.07502311, -0.06251926, -0.05001541,
       -0.03751155, -0.0250077 , -0.01250385,  0.        ,  0.01250385,
        0.0250077 ,  0.03751155,  0.05001541,  0.06251926,  0.07502311,
        0.08752696,  0.10003081,  0.11253466,  0.12503851,  0.13754236,
        0.15004622,  0.16255007,  0.17505392,  0.18755777,  0.20006162,
        0.21256547,  0.22506932,  0.23757318,  0.25007703,  0.26258088,
        0.27508473,  0.28758858,  0.30009243,  0.31259628,  0.32510014,
        0.33760399,  0.35010784,  0.36261169,  0.37511554,  0.38761939,
        0.40012324,  0.41262709])

In [278]:
ind_x

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False])

In [279]:
power_Spectrum

array([[[0.00040279, 0.00040793, 0.00042073, ..., 0.00042246,
         0.00040946, 0.00040328],
        [0.00041976, 0.00042473, 0.0004357 , ..., 0.00044328,
         0.00042796, 0.00042044],
        [0.00045858, 0.00046229, 0.00047428, ..., 0.00048668,
         0.00046972, 0.00046067],
        ...,
        [0.00046067, 0.00046972, 0.00048668, ..., 0.00047428,
         0.00046229, 0.00045858],
        [0.00042044, 0.00042796, 0.00044328, ..., 0.0004357 ,
         0.00042473, 0.00041976],
        [0.00040328, 0.00040946, 0.00042246, ..., 0.00042073,
         0.00040793, 0.00040279]],

       [[0.00040358, 0.00040891, 0.00042029, ..., 0.00042204,
         0.00040947, 0.00040321],
        [0.00041955, 0.00042396, 0.00043643, ..., 0.000442  ,
         0.00042835, 0.00042088],
        [0.00045778, 0.00046224, 0.00047361, ..., 0.0004861 ,
         0.00046907, 0.00046065],
        ...,
        [0.00046172, 0.00047069, 0.00048707, ..., 0.0004741 ,
         0.00046269, 0.00045875],
        [0.0

In [280]:
# Use boolean indexing to extract the desired values
power_Spectrum_cut = power_Spectrum[ind_w, ind_x, ind_y]

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (598,) (9,) (9,) 

In [281]:
# Assuming ind_x, ind_y, and ind_w are your boolean arrays
result_w = power_Spectrum[ind_w, :, :]
result_x = power_Spectrum[:, ind_x, :]
result_y = power_Spectrum[:, :, ind_y]


In [282]:
result_x.shape

(1200, 9, 67)

In [261]:
power_Spectrum[598, 9, 9]

0.02133856703593532

In [284]:
# Get index boundaries
ind_x = np.abs(Kx) <= K_limits[1]
ind_y = np.abs(Ky) <= K_limits[1]
ind_w = (w >= W_limits[0]) & (w <= W_limits[1])

# Create 3D structure using meshgrid
W_3D, Kx_3D, Ky_3D = np.meshgrid(w[ind_w], Kx[ind_x], Ky[ind_y], indexing='ij')

# Use boolean indexing to extract the desired values
power_Spectrum_cut = power_Spectrum[ind_w, ind_x, ind_y]


IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (598,) (9,) (9,) 

In [286]:
# Get index boundaries
ind_x = np.abs(Kx) <= K_limits[1]
ind_y = np.abs(Ky) <= K_limits[1]
ind_w = (w >= W_limits[0]) & (w <= W_limits[1])

# Create 3D structure using meshgrid
W_3D, Kx_3D, Ky_3D = np.meshgrid(w[ind_w], Kx[ind_x], Ky[ind_y], indexing='ij')

# Use np.ix_ for boolean indexing to extract the desired values
power_Spectrum_cut = power_Spectrum[np.ix_(ind_w, ind_x, ind_y)]


In [288]:
power_Spectrum_cut.shape

(598, 9, 9)

In [336]:
# Calculate the magnitude of K_3D
K_3D = np.sqrt(Kx_3D**2 + Ky_3D**2)

# Set to NaN values outside K_limits
power_Spectrum_cut[(K_3D < K_limits[0]) | (K_3D > K_limits[1])] = np.nan

In [337]:
power_Spectrum_cut

array([[[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,  401.68787517, ...,
          123.73874401,           nan,           nan],
        [          nan, 1556.17108338, 1233.40510308, ...,
          342.99381488,  704.96540765,           nan],
        ...,
        [          nan,  658.56380426,  316.49859469, ...,
         1324.88973102, 1655.83691818,           nan],
        [          nan,           nan,  116.96122784, ...,
          434.49822684,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan]],

       [[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,  393.97232223, ...,
          125.55662035,           nan,           nan],
        [          nan, 1532.53302761, 1211.91667607, ...,
          350.

In [262]:
# Get power spectra
Spectrum_raw = fftshift(fftn(IMG_3D) / np.prod(IMG_3D.shape))
power_Spectrum = np.abs(Spectrum_raw / Norm)**2

# Get index boundaries
ind_x = np.abs(Kx) <= K_limits[1]
ind_y = np.abs(Ky) <= K_limits[1]
ind_w = (w >= W_limits[0]) & (w <= W_limits[1])

# Create 3D structure using meshgrid
W_3D, Kx_3D, Ky_3D = np.meshgrid(w[ind_w], Kx[ind_x], Ky[ind_y], indexing='ij')
    
# Use boolean indexing to extract the desired values
power_Spectrum_cut = power_Spectrum[ind_w, ind_x, ind_y]

# Set to NaN values outside K_limits
K_3D = np.sqrt(Kx_3D**2 + Ky_3D**2)
power_Spectrum_cut[(K_3D < K_limits[0]) | (K_3D > K_limits[1])] = np.nan

# Get original spectrum limits
Kx_orig_limits = [Kx[0], Kx[-1]]
Ky_orig_limits = [Ky[0], Ky[-1]]
W_orig_limits = [w[0], w[-1]]

NameError: name 'IMG_3D' is not defined

In [172]:
K_limits = [-np.inf, np.inf]